# Azure嵌入示例

本示例将涵盖使用Azure OpenAI服务的嵌入。


## 设置

首先，我们安装必要的依赖项并导入我们将要使用的库。


In [ ]:
! pip install "openai>=1.0.0,<2.0.0"
! pip install python-dotenv


In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()


### 认证

Azure OpenAI 服务支持多种认证机制，包括 API 密钥和 Azure Active Directory 令牌凭据。


In [2]:
use_azure_active_directory = False  # 将此标志设置为 True，如果您正在使用 Azure Active Directory。


#### 使用API密钥进行身份验证

要设置OpenAI SDK以使用*Azure API密钥*，我们需要将`api_key`设置为与您的端点关联的密钥（您可以在[Azure门户](https://portal.azure.com)的*"资源管理"*下的*"密钥和端点"*中找到此密钥）。您还将在此处找到您资源的端点。


In [3]:
if not use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=api_key,
        api_version="2023-09-01-preview"
    )


#### 使用Azure Active Directory进行身份验证
现在让我们看看如何通过Azure Active Directory进行身份验证。我们将从安装`azure-identity`库开始。这个库将提供我们需要进行身份验证的令牌凭据，并帮助我们通过`get_bearer_token_provider`辅助函数构建一个令牌凭据提供程序。建议使用`get_bearer_token_provider`而不是向`AzureOpenAI`提供静态令牌，因为这个API会自动为您缓存和刷新令牌。

有关如何设置Azure Active Directory身份验证与Azure OpenAI的更多信息，请参阅[文档](https://learn.microsoft.com/azure/ai-services/openai/how-to/managed-identity)。


In [ ]:
! pip install "azure-identity>=1.15.0"


In [5]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

if use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        azure_ad_token_provider=get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
        api_version="2023-09-01-preview"
    )


> 注意：如果未提供以下参数，则AzureOpenAI将从它们对应的环境变量中推断出来：

- `api_key` 从 `AZURE_OPENAI_API_KEY`
- `azure_ad_token` 从 `AZURE_OPENAI_AD_TOKEN`
- `api_version` 从 `OPENAI_API_VERSION`
- `azure_endpoint` 从 `AZURE_OPENAI_ENDPOINT`


## 部署

在本节中，我们将创建一个模型的部署，以便我们可以用来创建嵌入。


### 部署：在Azure OpenAI Studio中创建
让我们部署一个模型以用于嵌入。前往 https://portal.azure.com，找到您的Azure OpenAI资源，然后导航到Azure OpenAI Studio。点击“部署”选项卡，然后为您想要用于嵌入的模型创建一个部署。您在模型中给出的部署名称将在下面的代码中使用。


In [4]:
deployment = "" # 在此处填写从门户获取的部署名称


## 嵌入

现在让我们使用我们构建的客户端来创建嵌入。


In [ ]:
embeddings = client.embeddings.create(
    model=deployment,
    input="The food was delicious and the waiter..."
)
                                
print(embeddings)
